# 다트API

*api key는 본인이 발급받은 key가 필요함

In [1]:
import requests
import tqdm

api_key = '3ca9004a926faf0344572c21df9df1107c2f626b'

### 기업명-기업코드 읽어오기

In [2]:
# 고유번호
import zipfile
from io import BytesIO

url_unique = "https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={}"
resq = requests.get(url_unique.format(api_key))
z = zipfile.ZipFile(BytesIO(resq.content))

In [3]:
z.namelist()

['CORPCODE.xml']

In [4]:
import xmltodict
import json

data_xml = z.read('CORPCODE.xml').decode('utf-8')
data_odict = xmltodict.parse(data_xml)
data_dict = json.loads(json.dumps(data_odict))
data = data_dict.get('result', {}).get('list')

In [5]:
# 상장사만 #code_dict에 저장되어 있음
code_list = []
for d in data:
    if d['stock_code'] != None:
        code_list.append([d['corp_code'], d['corp_name'],d['stock_code']])

import pandas as pd

df_corp = pd.DataFrame(data=code_list)
df_corp.columns = ['corp_code','corp_name','stock_code']

In [6]:
df_corp['stock_code'] = "A" + df_corp['stock_code'].apply(lambda x: x.zfill(6))
df_corp.columns = ["Dart코드","회사명","기업공시코드"]
df_corp = df_corp.sort_values("기업공시코드").reset_index(drop=True)

In [7]:
df_corp.to_csv("기업명_기업코드.csv")
df_corp.head()

,Dart코드,회사명,기업공시코드
0,00149293,신한은행,A000010
1,00119195,동화약품,A000020
2,00254045,우리은행,A000030
3,00112378,KR모터스,A000040
4,00101628,경방,A000050


### 날짜 인덱스 생성

In [8]:
## 날짜 인덱스 생성하기 (3달 window로 생성)

# 시작 인덱스
date_list = pd.date_range('20150101', '20160101', freq='3MS').astype('str').tolist()
start_date_list = []
for d in date_list:
    start_date_list.append(d.replace('-',''))

# 종료 인덱스
date_list2= pd.date_range('20150301', '20160101', freq='3M').astype('str').tolist()
end_date_list = []
for d in date_list2:
    end_date_list.append(d.replace('-',''))
end_date_list.append('20160331') # 맨 끝 하나 추가

final_date_list = list(zip(start_date_list, end_date_list))

### 공시검색 개발가이드

* 모든 공시를 다가져온다

In [9]:
# 공시검색 개발가이드 (공시정보)
url = "https://opendart.fss.or.kr/api/list.json"
l_data = []

for start, end in tqdm.tqdm(final_date_list):
    param_dict = {'crtfc_key': api_key,
              'bgn_de' : start, 'end_de': end,
              'pblntf_ty': 'B',
              'pblntf_detail_ty':'B001', 'corp_cls':'K',
              'page_count':'100'}

    req = requests.get(url, params=param_dict)
    data_json = req.json()
    # print(data_json)

    for dict in data_json['list']:
       l_data.append([dict['corp_code'], dict['corp_name'], dict['stock_code'], dict['rcept_dt'], dict['report_nm']])

l_data[:5]

100%|██████████| 5/5 [00:01<00:00,  3.48it/s]


[['00260611', '크로바하이텍', '043590', '20150309', '[첨부추가]주요사항보고서(자기주식취득신탁계약체결결정)'],
 ['00121686', '엠케이전자', '033160', '20150127', '[첨부추가]주요사항보고서(자기주식취득신탁계약체결결정)'],
 ['00136642', '엠벤처투자', '019590', '20150331', '[기재정정]주요사항보고서(전환사채권발행결정)'],
 ['01046708', '바디텍메드', '206640', '20150331', '주요사항보고서(합병결정)'],
 ['00370200', '와이오엠', '066430', '20150331', '[기재정정]주요사항보고서(중요한자산양수도결정)']]

In [11]:
from tqdm.contrib import tzip
import time

In [12]:
# 특정회사의 기간동안의 모든 공시 다 가져오기
param = {"crtfc_key":api_key,
         "corp_code":"115450",
         "bgn_de":"20220101",
         "end_de":"20230107"}

req = requests.get(url, params=param)
data_json = req.json()

print(data_json)

{'status': '000', 'message': '정상', 'page_no': 1, 'page_count': 10, 'total_count': 25, 'total_page': 3, 'list': [{'corp_code': '00552859', 'corp_name': 'HLB테라퓨틱스', 'stock_code': '115450', 'corp_cls': 'K', 'report_nm': '주요사항보고서(유상증자결정)', 'rcept_no': '20230104000128', 'flr_nm': 'HLB테라퓨틱스', 'rcept_dt': '20230104', 'rm': '정'}, {'corp_code': '00552859', 'corp_name': 'HLB테라퓨틱스', 'stock_code': '115450', 'corp_cls': 'K', 'report_nm': '전환청구권행사(제14회차)', 'rcept_no': '20230102900801', 'flr_nm': 'HLB테라퓨틱스', 'rcept_dt': '20230102', 'rm': '코정'}, {'corp_code': '00552859', 'corp_name': 'HLB테라퓨틱스', 'stock_code': '115450', 'corp_cls': 'K', 'report_nm': '[기재정정]주식배당결정', 'rcept_no': '20221228900344', 'flr_nm': 'HLB테라퓨틱스', 'rcept_dt': '20221228', 'rm': '코'}, {'corp_code': '00552859', 'corp_name': 'HLB테라퓨틱스', 'stock_code': '115450', 'corp_cls': 'K', 'report_nm': '배당락(주식배당)', 'rcept_no': '20221227900898', 'flr_nm': '코스닥시장본부', 'rcept_dt': '20221227', 'rm': '코'}, {'corp_code': '00552859', 'corp_name': 'HLB테라퓨틱스',

### 유상증자 개발가이드

In [15]:
stock_code_list = df_corp["Dart코드"].values

In [16]:
url = "https://opendart.fss.or.kr/api/piicDecsn.json"
l = []

for code, count in tzip(stock_code_list[700:800], range(len(stock_code_list[700:800]))):
    param = {"crtfc_key":api_key,
             "corp_code":code,
             "bgn_de":"20220101",
             "end_de":"20221231"}
    
    req = requests.get(url, params=param)
    data_json = req.json()

    if data_json['status'] == '000':
        l.append(data_json["list"])
        print(data_json)
    if int(count) == 20:
        time.sleep(1)

  0%|          | 0/100 [00:00<?, ?it/s]

{'status': '000', 'message': '정상', 'list': [{'rcept_no': '20221031000397', 'corp_cls': 'Y', 'corp_code': '00156150', 'corp_name': '하이트론씨스템즈', 'nstk_ostk_cnt': '2,148,995', 'nstk_estk_cnt': '-', 'fv_ps': '2,500', 'bfic_tisstk_ostk': '9,611,224', 'bfic_tisstk_estk': '-', 'fdpp_fclt': '-', 'fdpp_bsninh': '-', 'fdpp_op': '-', 'fdpp_dtrp': '-', 'fdpp_ocsa': '5,999,994,040', 'fdpp_etc': '-', 'ic_mthn': '제3자배정증자', 'ssl_at': 'N', 'ssl_bgd': '-', 'ssl_edd': '-'}]}
{'status': '000', 'message': '정상', 'list': [{'rcept_no': '20230117000277', 'corp_cls': 'K', 'corp_code': '00110750', 'corp_name': '리더스 기술투자', 'nstk_ostk_cnt': '23,000,000', 'nstk_estk_cnt': '-', 'fv_ps': '500', 'bfic_tisstk_ostk': '130,043,703', 'bfic_tisstk_estk': '-', 'fdpp_fclt': '-', 'fdpp_bsninh': '-', 'fdpp_op': '11,500,000,000', 'fdpp_dtrp': '-', 'fdpp_ocsa': '-', 'fdpp_etc': '-', 'ic_mthn': '제3자배정증자', 'ssl_at': 'N', 'ssl_bgd': '-', 'ssl_edd': '-'}]}
{'status': '000', 'message': '정상', 'list': [{'rcept_no': '20221208000503', 'co

### 종목코드 불러오기

In [8]:
df_corp.head()

,Dart코드,회사명,기업공시코드
0,00149293,신한은행,A000010
1,00119195,동화약품,A000020
2,00254045,우리은행,A000030
3,00112378,KR모터스,A000040
4,00101628,경방,A000050


In [9]:
import time
import pandas as pd
from tqdm import tqdm

url_code = "https://opendart.fss.or.kr/api/company.json"
return_dict = {}

for i,key in enumerate(tqdm(df_corp["Dart코드"].values)):
    param = {"crtfc_key":api_key,
             "corp_code":key}
    if i%20 == 0:
        time.sleep(2)
    req = requests.get(url=url_code, 
                       params=param).json()
    if req["status"] != "000":
        print(i, key, "오류발생!!")
    return_dict[req["stock_code"]] = req["corp_cls"]

pd.Series(return_dict).to_csv("기업코드_시장.csv")

100%|██████████| 3534/3534 [12:52<00:00,  4.57it/s]


In [23]:
s = pd.read_csv("기업코드_시장.csv")

s.columns = ["기업공시코드","시장"]

s["기업공시코드"] = s["기업공시코드"].apply(lambda x:"A" + str(x).zfill(6))

s1 = s.loc[s["시장"] == "Y"]
s2 = s.loc[s["시장"] == "K"]

pd.concat([s1,s2]).sort_values("기업공시코드").reset_index(drop=True).to_csv("기업코드_시장.csv")